# Tratamiento de los datos missing

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En concreto, **en muchos conjuntos de datos interesantes faltan algunos datos**.
Para complicar aún más las cosas, las diferentes fuentes de datos pueden indicar los datos que faltan de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos que faltan, discutiremos cómo Pandas elige representarlos, y demostraremos algunas herramientas incorporadas de Pandas para manejar los datos que faltan en Python.
**Aquí y a lo largo del libro, nos referiremos a los datos perdidos en general como valores *nulos*, *NaN*, o *NA***

## Compromisos en las convenciones sobre datos missing

Se han desarrollado varios esquemas para indicar la presencia de datos que faltan en una tabla o DataFrame.
Generalmente, giran en torno a una de dos estrategias: **utilizar una *máscara* que indica globalmente los valores que faltan, o elegir un *valor centinela* que indica una entrada que falta**.

En el enfoque de enmascaramiento, **la máscara** puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque de centinela, el valor centinela **puede ser alguna convención específica de los datos, como indicar que falta un valor entero con -9999** o algún patrón de bits raro, o puede ser una convención más global, como indicar que falta un valor de coma flotante con **NaN (Not a Number)**, un valor especial que forma parte de la especificación IEEE de coma flotante.

Ninguno de estos enfoques está exento de inconvenientes: el uso de una matriz de máscaras separada requiere la asignación de una matriz booleana adicional, lo que añade sobrecarga tanto de almacenamiento como de cálculo. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de la CPU y la GPU. Los valores especiales comunes, como NaN, no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, los distintos lenguajes y sistemas utilizan convenciones diferentes.
**Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican que faltan datos**, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda que indica un estado NA.

## Missing en Pandas

La forma en que Pandas **maneja los valores perdidos está limitada por su dependencia del paquete NumPy**, que no tiene una noción incorporada de valores NA para tipos de datos que no sean de coma flotante.

Pandas podría haber seguido el ejemplo de **R especificando patrones de bits para cada tipo de datos individual** para indicar nulidad, pero este enfoque resulta ser bastante difícil de manejar.
Mientras que R contiene cuatro tipos de datos básicos, **NumPy soporta *mucho* más que esto**: por ejemplo, mientras que R tiene un único tipo entero, NumPy soporta *catorce* tipos enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la signatura y la endianidad de la codificación.
**Reservar un patrón de bits específico en todos los tipos NumPy disponibles llevaría a una cantidad inmanejable de sobrecarga** en operaciones especiales para varios tipos, probablemente incluso requiriendo una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para utilizarlo como máscara reducirá significativamente el rango de valores que puede representar.

**NumPy tiene soporte para arrays enmascarados** - es decir, arrays que tienen un array booleano de máscara separado para marcar los datos como "buenos" o "malos".
Pandas podría haber derivado de esto, pero la sobrecarga en almacenamiento, cálculo y mantenimiento de código hace que sea una opción poco atractiva.

Con estas limitaciones en mente, **Pandas optó por utilizar centinelas para los datos que faltan**, y además optó por utilizar dos valores nulos ya existentes en Python: el valor especial de punto flotante **``NaN``, y el objeto Python ``None``**.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

> ## APUNTES XIME:

**CONVENCIONES SOBRE VALORES NULOS:**
- **NO se eliminan: ni filas ni columnas por tener algunos o datos nulos.** 

**SOLUCIÓN:** 
Debo imputarle algún dato según tipo de datos que se trate y que sea coherente para que no altere los demás datos. Hay dos opciones:

- **OPCIÓN 1 =** imputarle la mediana o la medio según que tipo de datos tenga.
  
***Por ejemplo***, si tengo una columna con la **edad de personas hombre** y faltan datos, no puedo tomar la media, porque nadie tiene 29.5 años, tengo una **variable discreta** (NO continua) entonces ahí, no corresponde la media, si no la **mediana**, para que no me modifique el análisis de los datos. SIEMPRE ver qué decisión tomar dependiendo los datos que se traten para que sea coherente el cálculo. 

>* Datos con variable discreta (enteros) = se suele tomar la MEDIANA
>* Datos con variable continua (decimales)= se suele tomar la MEDIA


- **OPCIÓN 2 =** Colocar un número absurdo. 
  
***Por ejemplo:*** poner el valor -9999, o si es un float 9.999. Luego hay que analizar como afecta ese valor en mis datos, la distribución, etc, pero lo mas importante es NO tener datos vacíos y hay que encontrar la forma de solventar ese vacío. 


>> ## ``None``: Datos Pythonic que faltan

El primer valor centinela utilizado por Pandas es **``None``**, un objeto Python singleton que se utiliza a menudo para los datos que faltan en el código Python.
Debido a que es un objeto Python, ``None`` no se puede utilizar en cualquier array arbitrario de NumPy/Pandas, sino sólo en arrays con tipo de datos ``'object'`` (es decir, arrays de objetos Python):

In [2]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

- `None`: los reconoce como tipo `object` > NO los reconoce como `Nonetype`.
- `NaN` de Numpy: los reconoce como `float` > ahí si se puede sumar sin error por ejemplo, pero siempre devuelve `nan`

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy puede inferir para los contenidos del array es que son objetos Python.
Aunque este tipo de array objeto es útil para algunos propósitos, cualquier operación sobre los datos se hará a nivel Python, con mucha más sobrecarga que las típicamente rápidas operaciones vistas para arrays con tipos nativos:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    # %timeit np.arange(1E6, dtype=dtype).sum()
    # print()

dtype = object
dtype = int


El uso de objetos Python en un array también significa que si realizas agregaciones como ``sum()`` o ``min()`` a través de un array con un valor ``None``, generalmente obtendrás un error:

In [ ]:
vals1.sum() # ERROR = PORQUE DICE QUE NO SE PUEDE SUMAR ENTEROS CON NONETYPE

> **VER TIPO DE DATOS en un DataFrame**:

Para averiguar el tipo de datos de un DataFrame, podemos hacer dos cosas:
- `pd.DataFrame(....).info()` > Al final de todo agregandole .info(), te devuelve info del DF, incluido el tipo de datos.
- `df.dtypes` > Devuelve el tipo de datos de cada clave

Vemos que, **los valores int por tener un None, devuelve que son FLOAT:**

In [9]:
a = pd.DataFrame({ 'int': [1,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})

a.dtypes

int       float64
float     float64
object     object
dtype: object

> **CAMBIAR TIPO DE DATOS**:
Si queremos cambiar el tipo de datos hay varias opciones:
- `a['int'] = a['int'].astype(pd.Int32Dtype())` > SIEMPRE PARENTESIS porque es un método de Pandas el dtype!!
- `a['int'] = a['int'].astype('Int32')` > Int, primera letra siempre mayusculas porque es de Pandas, para diferenciarlo con el de Numpy que no va con mayúculas.

Observar que al cambiar el tipo de datos, ahora no es más float, si no lo que le digo, pero en el DataFrame **figura como NA**, y no como antes: NaN

In [16]:
# OPCIÓN 1
a['int'] = a['int'].astype(pd.Int32Dtype()) # pd.Int32Dtype() > SIEMPRE PARENTESIS!! 
a.dtypes
print(a)

    int  float object
0     1    1.1      a
1     2    2.2     bb
2     3    3.3    ccc
3  <NA>    NaN   None


In [14]:
# OPCIÓN 2
a['int'] = a['int'].astype('Int32') # Int en MAYUSCULAS pq es de Pandas! Si no da error
a

,int,float,object
0,1,1.1,a
1,2,2.2,bb
2,3,3.3,ccc
3,<NA>,NaN,None


Esto refleja el hecho de que la suma entre un entero y ``None`` es indefinida.

>> ## ``NaN``: Missing numerical data

La otra representación de datos que faltan, ``NaN`` (acrónimo de *Not a Number*), es diferente; se trata de un valor especial de coma flotante reconocido por todos los sistemas que utilizan la representación estándar IEEE de coma flotante:

In [17]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Observa que NumPy eligió un tipo nativo de coma flotante para este array: esto significa que **a diferencia del array `object` de antes, este array soporta operaciones rápidas introducidas en código compilado.**

- `None`: los reconoce como tipo `object` > NO los reconoce como `Nonetype`.
- `NaN` de Numpy: los reconoce como `float` > ahí si se puede sumar sin error por ejemplo, pero siempre devuelve `nan`

Debes tener en cuenta que ``NaN`` es un poco como un virus de datos - infecta cualquier otro objeto que toca. Independientemente de la operación, **el resultado de la aritmética con ``NaN`` será otro ``NaN``**:

In [18]:
1 + np.nan

nan

In [19]:
0 *  np.nan

nan

In [20]:
vals2

array([ 1., nan,  3.,  4.])

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a error) pero **NO** siempre son útiles:

In [21]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

>> ##### NumPy proporciona algunas agregaciones especiales que ignorarán estos valores perdidos:

- **np.nansum** = SUMA de elementos con nan 
- **np.nanmin** = MINIMA de elementos con nan 
- **np.nanmax** = MAXIMA de elementos con nan 
- **np.nanmean** = MEDIA de elementos con nan > ACÁ NO TIENE EN CUENTA EL VALOR FALTANTE, divide solamente por los números reales, lo cual no está bien. 

In [23]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2), np.nanmean(vals2)

(8.0, 1.0, 4.0, 2.6666666666666665)

Tenga en cuenta que ``NaN`` es específicamente un **valor de coma flotante**; NO existe un valor NaN equivalente para enteros, cadenas u otros tipos.

>> ## NaN and None in Pandas

Tanto ``NaN`` como ``None`` tienen su lugar, y Pandas está construido para manejar los dos casi indistintamente, convirtiendo entre ellos cuando sea apropiado. 

* Al ser **PANDAS** (diferente con Numpy) > Si hay un `NaN` y un `None` a la vez, **todos los valores faltantes van a ser `NaN`**, porque tiene más peso que el None (como en el caso de los strings y enteros por ejemplo, que tiene más valor los strings)

In [24]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para los tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA presentes.
Por ejemplo, **si establecemos un valor en un array de enteros a ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA**:

In [25]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [27]:
x[0] = None # Cambia el tipo de dato a float y None, se convierte en NaN
x

0    NaN
1    1.0
dtype: float64

Observe que **además de convertir el array de enteros a coma flotante, Pandas convierte automáticamente el valor ``None`` a un valor ``NaN``**.
(Tenga en cuenta que hay una propuesta para añadir un entero nativo NA a Pandas en el futuro; en el momento de escribir esto, no se ha incluido).

Mientras que este tipo de magia puede parecer un poco "hackish" en comparación con el enfoque más unificado de los valores NA en lenguajes de dominio específico como R, el enfoque centinela/casting de Pandas funciona bastante bien en la práctica y en mi experiencia sólo rara vez causa problemas.

La siguiente tabla lista las convenciones de upcasting en Pandas cuando se introducen valores NA:

| Typeclass     | Conversión al almacenar NA | NA Valor centinela     |
|--------------|-----------------------------|------------------------|
| ``floating`` | Sin cambios                   | ``np.nan``             |
| ``object``   | Sin cambios                   | ``None`` or ``np.nan`` |
| ``integer``  | Cambia a ``float64``         | ``np.nan``             |
| ``boolean``  | Cambia a ``object``          | ``None`` or ``np.nan`` |

Tenga en cuenta que en Pandas, los **datos de cadena** siempre se almacenan con un dtype ``object``.

>> ## Operar con valores nulos

Como hemos visto, **Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores perdidos o nulos.**
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Estos son:

- ``isnull()``: Genera una máscara booleana que indica valores perdidos
- ``notnull()``: Lo contrario de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores perdidos rellenados o imputados

Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

> ### Detección de valores nulos = ``isnull()`` y ``notnull()``
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``.
Cualquiera de ellos devolverá una máscara booleana sobre los datos.

>> #### ``isnull()``:

In [28]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [29]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

Con un DataFrame + `isnull()`:

In [39]:
a = pd.DataFrame({ 'int': [1,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a[a['int'].isnull()]

,int,float,object
3,NaN,NaN,None


Si pongo un None tambien en la posición 0, devuelve solo esas filas:

In [44]:
a = pd.DataFrame({ 'int': [1,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a[a['int'].isnull()]

,int,float,object
3,NaN,NaN,None


In [52]:
a.isnull()

,int,float,object
0,True,False,False
1,False,False,False
2,False,False,False
3,True,True,True


>> #### ``notnull()``:

In [30]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

Como se menciona en [Data Indexing and Selection](2_Data-Indexing-and-Selection.ipynb), las máscaras booleanas se pueden utilizar directamente como un índice ``Series`` o ``DataFrame``:

In [45]:
data[data.notnull()]

0        1
2    hello
dtype: object

Con un DataFrame + `notnull()`:

In [3]:
a = pd.DataFrame({ 'int': [1,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a[a['int'].notnull()]

,int,float,object
0,1.0,1.1,a
1,2.0,2.2,bb
2,3.0,3.3,ccc


- Si cambio el primer 1 de 'int' por un None, no devuelve la fila 0 ni la 3, porque en esas hay None, o sea, datos nulos:

In [49]:
a = pd.DataFrame({ 'int': [None,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a[a['int'].notnull()]

,int,float,object
1,2.0,2.2,bb
2,3.0,3.3,ccc


- Pero si elijo que me muestre la columna `'float'`, va a figurar el NaN de la columna `'int'` porque la máscara la estoy haciendo según la columna 'float' y ahí no hay ningún None. 

In [50]:
a = pd.DataFrame({ 'int': [None,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a[a['float'].notnull()]

,int,float,object
0,NaN,1.1,a
1,2.0,2.2,bb
2,3.0,3.3,ccc


In [51]:
a.notnull()

,int,float,object
0,False,True,True
1,True,True,True
2,True,True,True
3,False,False,False


In [ ]:
# data[~(data==1)]

In [ ]:
data[~data.isnull()]

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para los ``DataFrame``.

> ### Eliminación de valores nulos = ``dropna()``

Además del enmascaramiento utilizado anteriormente, existen los métodos ``dropna()`` (que elimina los valores nulos) y ``fillna()`` (que rellena los valores nulos).

No podemos eliminar valores individuales de un ``DataFrame``; sólo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, puede que quieras una cosa u otra, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Para una ``Serie``, el resultado es sencillo:

In [53]:
data.dropna()
# data.dropna(inplace=True)
# data = data.dropna()

0        1
2    hello
dtype: object

In [54]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

> **Con un DataFrame + `dropna()`:**

**Elimina por fila** > Si en la fila detecta un None, directamente elimina todos los datos.

- **El `axis=` por defecto es 0.** 
- Si pongo `axis=`1 > Elimina por columna. Pero si tengo un valor nulo en todas las columnas como sucede en el siguiente DataFrame, eliminará todas y devolverá sólo el índice. 

In [4]:
a = pd.DataFrame({ 'int': [None,2,3, None],
                  'float': [1.1,2.2,3.3, None],
                  'object': ['a', 'bb', 'ccc', None]})
a.dropna()

,int,float,object
1,2.0,2.2,bb
2,3.0,3.3,ccc


> **`dropna()` + how= `'all'` y `'any'` :**

- **`'all'`** = Elimina filas y no las devuelve, si TODOS son NaN. En el ejemplo de abajo, devuelve la fila 0 porque solo 1 un valor en NaN, no todos. En cambio, no devuelve la fila 3 porque ahí sí todos son NaN. 
- **`'any'`** =  eliminará TODAS las filas que contienen al menos un valor nulo, dejando solo las filas que no tienen ningún valor nulo en ninguna de sus columnas.En el ejemplo de abajo, solo deuvelve filas 1 y 2, porque NO tienen ningun valor nulo, las otras tienen al menos uno. 

`HOW` + `AXIS=` >> También puede combinarse con `axis=`, por defecto siempre lo hará por filas (`axis='rows'`), si quiero que lo haga por columnas pongo `axis=1` o `axis='columns'`

In [9]:
a.dropna(how='all')

,int,float,object
0,NaN,1.1,a
1,2.0,2.2,bb
2,3.0,3.3,ccc
3,NaN,NaN,None


In [11]:
a.dropna(how='any')

,int,float,object
1,2.0,2.2,bb
2,3.0,3.3,ccc


> **`dropna()` + `thresh=`:**

**`thresh=`** > Significa "umbral". Se usa para definir un umbral para el número mínimo de valores no nulos que deseas en tus datos, ya sea en filas o columnas. 

Devuelve la fila que cumpla con el **número** que le paso de **valores no nulos** que deben estar presentes en filas o columnas para que estas no sean eliminadas.

In [13]:
a.dropna(thresh=3)

,int,float,object
1,2.0,2.2,bb
2,3.0,3.3,ccc


Para un ``DataFrame``, hay más opciones.

Considera el siguiente ``DataFrame``:

In [44]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]], columns=list('abc'))
df

,a,b,c
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


> Para saber **cuántos valores NO nulos** hay por columna: `.info()` y `.notnull().sum()`

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       2 non-null      float64
 1   b       2 non-null      float64
 2   c       3 non-null      int64  
dtypes: float64(2), int64(1)
memory usage: 204.0 bytes


In [47]:
df.notnull().sum()

a    2
b    2
c    3
dtype: int64

* Para saber **cuántos valores NULOS** hay por columna: `.isnull().sum()`

In [46]:
df.isnull().sum()

a    1
b    1
c    0
dtype: int64

Pero esto también elimina algunos datos buenos; quizá le interese más eliminar filas o columnas con *todos* los valores NA, o con una mayoría de valores NA.
Esto se puede especificar mediante los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de nulos que se permiten.

El valor predeterminado es ``how='any'``, de forma que se descartará cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo.
También puede especificar ``how='all'``, que sólo eliminará las filas/columnas que sean *todos* valores nulos:

In [ ]:
df.dropna(axis='rows', thresh=len(df.columns)*0.75)

In [ ]:
df.dropna(axis='columns', thresh=len(df)*0.75)

Aquí se han eliminado la primera y la última fila, porque sólo contienen dos valores no nulos.

> ### **Rellenar valores nulos = `fillna()`**

A veces, en lugar de eliminar los valores nulos, prefiere sustituirlos por un valor válido.
Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación a partir de los valores buenos.
Esto se puede hacer in situ usando el método ``isnull()`` como máscara, pero como es una operación tan común Pandas proporciona el método ``fillna()``, que devuelve una copia del array con los valores nulos reemplazados.

Considera la siguiente ``Serie``:

In [15]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

> **`fillna()` + `inplace=True`:**
* **Siempre hay que pasarle un PARÁMETRO a `fillna()` con lo que quiero rellenar los valores nulos.**
* Si quiero que **MODIFIQUE** el DataFrame, **debo usar `inplace=True`** para **GUARDARLO** EN LA MISMA VARIABLE. Por defecto, inplace= viene False. 
Podemos rellenar las entradas NA con un único valor, como **cero**:

In [16]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Podemos especificar un forward-fill para propagar el valor anterior hacia adelante:

In [17]:
# forward-fill
data.ffill()

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O podemos especificar un back-fill para propagar los siguientes valores hacia atrás:

In [18]:
# back-fill
data.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para ``DataFrame``s, las opciones son similares, pero también podemos especificar un ``axis`` a lo largo del cual se producen los rellenos:

In [ ]:
df

In [ ]:
df[1]

In [ ]:
df.fillna(df[1].mean())

In [ ]:
df

In [ ]:
df.fillna(method='bfill', axis=1)

Tenga en cuenta que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.

>> ## `equals()` + `compare()`

- `equals()` = Devuelve True o False dependiendo si 2 DataFrame son iguales o diferentes. 
- `compare()` = Devuelve esa diferencia mediante una tabla. En 'self' pone la que primero se nombra y en 'other' la que se pone entre parentesis. Es muy útil para saber en qué son diferentes.

In [24]:
j = pd.DataFrame({ 'int': [1,2,3,4],
                  'float': [1.1,2.2,3.3,4.4],
                  'object': ['a', 'bb', 'ccc', 'dddd']})

r = pd.DataFrame({ 'int': [1,2,3,5],
                  'float': [1.1,2.2,3.3,5.5],
                  'object': ['a', 'bb', 'ccc', 'ddddd']})

In [25]:
j.equals(r)

False

In [26]:
j.compare(r)

int       float       object       
  self other  self other   self  other
3  4.0   5.0   4.4   5.5   dddd  ddddd

**IMPORTANTE!! >>** Si DOS tablas se ven IGUAL, pero con `equals()` dice que es False, y con `compare()` NO muestra la diferencia, lo que podría ocurrir es que los TIPOS DE DATOS sean DIFERENTES!! 

En el ejemplo, podríamos hacer lo siguiente para que nos muestre las diferencias en los tipos de datos:
`j.dtypes.compare(r.dtypes)` > y así COMPARO LOS TIPOS DE DATOS

In [33]:
j = pd.DataFrame({ 'int': [1,2,3,4],
                  'float': [1.1,2.2,3.3,4.4],
                  'object': ['a', 'bb', 'ccc', 'dddd']})

r = pd.DataFrame({ 'int': [1,2,3,4],
                  'float': [1.1,2.2,3.3,4.4],
                  'object': ['a', 'bb', 'ccc', 'dddd']})

j['int'] = j['int'].astype('Int32') # CAMBIO EL TIPO DE DATO DE LA COLUMNA 'Int'

In [34]:
j.equals(r)

False

In [35]:
j.compare(r) # NO muestra nada porque a simple vista son iguales.

Empty DataFrame
Columns: []
Index: []

In [36]:
j.dtypes.compare(r.dtypes)

,self,other
int,Int32,int64
